In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
!export PYSPARK_PYTHON=python3.6.9

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D
from tensorflow.keras.layers import MaxPooling2D, Dropout,Flatten
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
import subprocess
import tensorflow as tf
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from nltk.corpus import brown
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
import pyspark.sql.functions as f
import json
import re
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import sys
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import lit
import pickle
import tensorflow.keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from keras.models import model_from_json
from tensorflow import keras
from pyspark.sql.functions import udf
import pandas as pd

spark.sparkContext.getConf().getAll()

Using TensorFlow backend.


[('spark.eventLog.enabled', 'true'),
 ('spark.executor.instances', '3'),
 ('spark.sql.execution.arrow.enabled', 'true'),
 ('spark.driver.memory', '4g'),
 ('spark.executor.cores', '5'),
 ('spark.executor.extraJavaOptions', '-XX:+UseG1GC'),
 ('spark.driver.extraJavaOptions', '-XX:+UseG1GC'),
 ('spark.driver.host', 'kafka1'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.yarn.archive', 'hdfs:///user/spark/conf/spark-libs.jar'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.sql.shuffle.partitions', '200'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spark.kryoserializer.buffer.max', '2047'),
 ('spark.executor.memoryOverhead', '1g'),
 ('spark.driver.memoryOverhead', '1g'),
 ('spark.executor.memory', '4g'),
 ('spark.executor.id', 'driver'),
 ('spark.history.fs.logDirectory', 'file:///root/spark/eventLog'),
 ('spark.default.parallelism', '163'),
 ('spark.app.name', 

In [3]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
import pyspark.sql.functions as f
import json
import re
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import sys
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import lit

import pickle
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from keras.models import model_from_json

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D
from keras.layers import MaxPooling2D, Dropout,Flatten
from keras import backend as K
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
import subprocess
import nltk

In [5]:
nltk.download('brown')

[nltk_data] Error loading brown: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


False

In [6]:
spark

In [7]:
sc = spark._jsc.sc() 

In [8]:
sc

JavaObject id=o165

In [9]:
print(sc.version)

In [10]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
import pyspark.sql.functions as f
import json
import re
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import sys
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import lit

import pickle
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from keras.models import model_from_json

In [11]:
spark 

In [12]:
sc = spark.sparkContext

In [13]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [14]:
sql_context = SQLContext(sc)

In [15]:
sql_context

In [16]:
sc.master

'yarn'

# Data Preprocessing

In [17]:
import re

def regex_(text):
    # 영어, 숫자, 특수만문자 제외 삭제.  
    text = text.dropna(axis=0)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text['total_text_list'] = text['total_text_list'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['total_text_list'] = text['total_text_list'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['total_text_list'] = text['total_text_list'].str.replace(pat=pattern,repl=r'',regex=True)
    text['total_text_list'] = text['total_text_list'].str.replace(pat=r'(\@\w+.*?) ',repl=r'',regex=True)
    text['total_text_list'] = text['total_text_list'].str.replace(pat=r'[^ a-zA-Z]',repl=r'',regex=True)
    text['total_text_list'] = text['total_text_list'].str.lower()
    text=text[((text['total_text_list'].str.len()>= 10)) & ((text['total_text_list'].eq(' ')==False) & (text['total_text_list'].eq('')==False))]
    return text
def regex2_(text):
    # 영어, 숫자, 특수만문자 제외 삭제.  
    text = text.dropna(axis=0)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text['text'] = text['text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['text'] = text['text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['text'] = text['text'].str.replace(pat=pattern,repl=r'',regex=True)
    text['text'] = text['text'].str.replace(pat=r'(\@\w+.*?) ',repl=r'',regex=True)
    text['text'] = text['text'].str.replace(pat=r'[^ a-zA-Z]',repl=r'',regex=True)
    text['text'] = text['text'].str.lower()
    text=text[((text['text'].str.len()>= 10)) & ((text['text'].eq(' ')==False) & (text['text'].eq('')==False))]
    return text

def regex_3(text):
    # 영어, 숫자, 특수만문자 제외 삭제.  
    text = text.dropna(axis=0)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    text['full_text'] = text['full_text'].str.replace(pat=r'[^ a-zA-Z]',repl=r'',regex=True)
    # @로 시작하는 애들도 정규식 이용해서 없애기 @[^ ]+
    text['full_text'] = text['full_text'].str.replace(pat=r'(\@\w+.*?) ',repl=r'',regex=True)
    text['full_text'] = text['full_text'].str.lower()
    text=text[((text['full_text'].str.len()>= 10)) & ((text['full_text'].str.split(" ").str.len()>= 10)) & ((text['full_text'].eq(' ')==False) & (text['full_text'].eq('')==False))]
    return text

def regex_4(text):
    # 영어, 숫자, 특수만문자 제외 삭제.  
    text = text.dropna(axis=0)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text['full_text'] = text['full_text'].str.replace(pat=pattern,repl=r'',regex=True)
    text['full_text'] = text['full_text'].str.replace(pat=r'[^ a-zA-Z]',repl=r'',regex=True)
    # @로 시작하는 애들도 정규식 이용해서 없애기 @[^ ]+
    text['full_text'] = text['full_text'].str.replace(pat=r'(\@\w+.*?) ',repl=r'',regex=True)
    text['full_text'] = text['full_text'].str.lower()
    #text=text[((text['full_text'].str.len()>= 10)) & ((text['full_text'].str.split(" ").str.len()>= 10)) & ((text['full_text'].eq(' ')==False) & (text['full_text'].eq('')==False))]
    return text

In [18]:
# general_minsun2.csv
general = pd.read_csv('/root/spark/bigcomp_model/general.csv', sep=',',  lineterminator='\n')
general = general.drop_duplicates()
general = spark.createDataFrame(general[['total_text_list']])
general = general.select(col('total_text_list').alias('text')) 
general = general.withColumn('label', lit(0))
general.show(5)
print(general.count())
general = general.withColumn("label",col("label").cast("integer"))
# 보안 관련 계정들 트윗
security = pd.read_csv('/root/spark/bigcomp_model/typoon_final.csv', sep=',',  lineterminator='\n')
security
security = spark.createDataFrame(security[['full_text']])
security = security.select(col('full_text').alias('text')) 
security = security.withColumn('label', lit(1))
security = security.withColumn("label",col("label").cast("integer"))
security.show(5)
print(security.count())
 # 2007~2019"

/root/spark/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  An error occurred while calling z:org.apache.spark.sql.api.python.PythonSQLUtils.readArrowStreamFromFile.
: java.lang.IllegalArgumentException
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:334)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.readMessage(MessageSerializer.java:543)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.readNextBatch(ArrowConverters.scala:243)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.<init>(ArrowConverters.scala:229)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$.getBatchesFromStream(ArrowConverters.scala:228)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun$readArrowStreamFromFile$2.apply(ArrowConverters.scala:216)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun

+--------------------+-----+
|                text|label|
+--------------------+-----+
|im at ap stgate i...|    0|
|why do i love thi...|    0|
|nowplaying charmi...|    0|
|im at kuwait city...|    0|
|lolim right hande...|    0|
+--------------------+-----+
only showing top 5 rows

801932
+--------------------+-----+
|                text|label|
+--------------------+-----+
|preparing for tro...|    1|
|weve got to prep ...|    1|
|dont know how muc...|    1|
|whoo hoo we are t...|    1|
|hurricane kay is ...|    1|
+--------------------+-----+
only showing top 5 rows

1640986


In [19]:
t05 = security.collect()[:500000]
t9 = general.collect()[:500000]
t05 = spark.createDataFrame(t05)
t9 = spark.createDataFrame(t9)
fifth = t05.union(t9)

In [20]:
t1 = security.collect()[200000:250000]
t2 = security.collect()[250000:300000]
t3 = security.collect()[300000:350000]
t4 = security.collect()[350000:400000]

t5 = general.collect()[200000:250000]
t6 = general.collect()[250000:300000]
t7 = general.collect()[300000:350000]
t8 = general.collect()[350000:400000]


t1 = spark.createDataFrame(t1)
t2 = spark.createDataFrame(t2)
t3 = spark.createDataFrame(t3)
t4 = spark.createDataFrame(t4)
t5 = spark.createDataFrame(t5)
t6 = spark.createDataFrame(t6)
t7 = spark.createDataFrame(t7)
t8 = spark.createDataFrame(t8)


first = t1.union(t5)
second = t1.union(t2).union(t5).union(t6)
third = t1.union(t2).union(t3).union(t5).union(t6).union(t7)
fourth = t1.union(t2).union(t3).union(t4).union(t5).union(t6).union(t7).union(t8)


df1 = t1.union(t5)
df2 = t2.union(t6)
df3 = t3.union(t7)
df4 = t4.union(t8)

word1 = security.collect()[200000:250000]
word2 = general.collect()[200000:250000]

word3 = security.collect()[200000:300000]
word4 = general.collect()[200000:300000]

word5 = security.collect()[200000:350000]
word6 = general.collect()[200000:350000]

word7 = security.collect()[200000:400000]
word8 = general.collect()[200000:400000]

word1 = spark.createDataFrame(word1)
word2= spark.createDataFrame(word2)

word3 = spark.createDataFrame(word3)
word4= spark.createDataFrame(word4)

word5 = spark.createDataFrame(word5)
word6= spark.createDataFrame(word6)

word7 = spark.createDataFrame(word7)
word8= spark.createDataFrame(word8)

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
def background_keyword(df): # input : spark dataframe을 pandas로 변환한 후, dataframe column
    background = df.values.tolist()
    token = Tokenizer()         # 토큰화 함수 지정
    token.fit_on_texts(background)    # 토큰화 함수에 문장 적용
    sorted_dic4 = sorted(token.word_counts.items(), key=lambda x: x[1], reverse=True)
    word_counts_1 = {}
    for i in sorted_dic4:
        word_counts_1.update({i[0]:i[1]})
    return word_counts_1 # dictionary 임
def back_keyword(pos,neg): # pos : event-related keyword, neg : event-unrelated keyword
    stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
    stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')
    word1 = stage_1.transform(pos) #나는 first_train 부터 함!
    word1 = stage_2.transform(word1)
    word2 = stage_1.transform(neg) #나는 first_train 부터 함!
    word2 = stage_2.transform(word2)
    from pyspark.sql.functions import col, concat_ws
    df1 = word1.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word1 = df1.select("filtered_words").toPandas()
    df2 = word2.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word2 = df2.select("filtered_words").toPandas()
    word_counts_1 = background_keyword(word1["filtered_words"])
    word_counts_2 = background_keyword(word2["filtered_words"])
    return word_counts_1, word_counts_2




# Author : Minseon Kim (2021)
class BroadcastWrapper(object):
    def __init__(self, data, token_list):
        self.broadcast_var = sc.broadcast(data)
#         self.last_updated_time = datetime.now()
        self.token_list = token_list
    
#     def is_should_be_updated(self, data):
#         cur_time = datetime.now()
#         diff_sec = (cur_time - self.last_updated_time).total_seconds()
#         return self.broadcast_var is None or diff_sec> 1
    
    def update_and_get_data(self, spark):
        a = self.broadcast_var.value
        self.broadcast_var.unpersist()
        for i in self.token_list:
            for j in i:
                if j not in a.keys():
                    a[j] = 1
                else:
                    a[j] += 1
        new_data = a
        self.broadcast_var = spark.broadcast(new_data)
#         self.last_updated_time = datetime.now()
#         return len(self.token_list)
        return self.broadcast_var

# 바꾼 버전
from pyspark.ml import Transformer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame
from pyspark.sql import Row
from pyspark.sql import types   

def series_to_list(x):
    seri = x.values.tolist()
    for i in range(len(seri)):
        seri[i] = seri[i].tolist()
    return seri
class TextToSequence(Transformer):
    w1 = dict()
    w2 = dict()
    pos_t = Tokenizer(lower=False)
    neg_t = Tokenizer(lower=False)
    pos_vocab = []
    neg_vocab = []
    
    def __init__(self, w1: Dict[str, int], w2:Dict[str, int]):
        super(TextToSequence, self).__init__()
        self.w1 = w1
        self.w2 = w2
        
    
    def _transform(self, df: DataFrame):
        result_pdf = df.select("filtered_words").toPandas()
        pos_df = result_pdf[:150000] # 24939 99756
        neg_df = result_pdf[150000:]
        print(len(result_pdf))
        
        broadcast_wrapper1 = BroadcastWrapper(self.w1, pos_df["filtered_words"])
        ww1 = broadcast_wrapper1.update_and_get_data(sc).value
        
        broadcast_wrapper2 = BroadcastWrapper(self.w2, neg_df["filtered_words"])
        ww2 = broadcast_wrapper2.update_and_get_data(sc).value
        
        www1 = {k: v for k, v in sorted(ww1.items(), key=lambda item: item[1], reverse=True)}
        www2 = {k: v for k, v in sorted(ww2.items(), key=lambda item: item[1], reverse=True)}
        print(www1['allow'])
        print(www2['like'])
        aa = [w for i, w in enumerate(www1) if i < 5000]
        bb = [w for i, w in enumerate(www2) if i < 5000]
        
        self.pos_t.fit_on_texts(aa)
        self.neg_t.fit_on_texts(bb)
#         encoded_docs_pos = self.pos_t.texts_to_sequences(df.select("filtered_words1").toPandas())
#         encoded_docs_neg = self.neg_t.texts_to_sequences(df.select("filtered_words1").toPandas())
        result_list = series_to_list(result_pdf["filtered_words"])
        encoded_docs_pos = self.pos_t.texts_to_sequences(result_list) # 상위 event keyword 5k가 쓰인 tokenizer
        encoded_docs_neg = self.neg_t.texts_to_sequences(result_list)
        
        X_p = pad_sequences(encoded_docs_pos, maxlen=100, padding='post').tolist()
        X_n = pad_sequences(encoded_docs_neg, maxlen=100, padding='post').tolist()
        
        text_array = [str(row['text']) for row in df.select('text').collect()]
        label_array = [int(row['label']) for row in df.select('label').collect()]

        
        zip_array = list(zip(text_array, label_array, X_p, X_n))
        rdd = sc.parallelize(zip_array, numSlices=306)
        fdf = rdd.toDF(['text','label','feature1','feature2'])
        

        print(type(fdf))

        return fdf, www1, aa
    
    
word_counts_1, word_counts_2 = back_keyword(word1, word2)
word_counts_1 = sc.broadcast(word_counts_1)
word_counts_2 = sc.broadcast(word_counts_2)

stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')

r1 = stage_1.transform(df1) #나는 first_train 부터 함!
r2 = stage_2.transform(r1)


# USE THE TRANSFORMER WITHOUT PIPELINE
text_sequence = TextToSequence(w1 = word_counts_1.value, w2 = word_counts_2.value)
df_example, www2, bb = text_sequence.transform(r2)
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df_with_vectors = df_example.select(
    df_example["label"], 
    list_to_vector_udf(df_example["feature1"]).alias("feature1"), 
    list_to_vector_udf(df_example["feature2"]).alias("feature2")
)
label_str_index = StringIndexer(inputCol='label', outputCol='label_index') # transformer의 마지막 단계!!
label_df = label_str_index.fit(df_with_vectors).transform(df_with_vectors)


100000
105
4221
<class 'pyspark.sql.dataframe.DataFrame'>


In [23]:
from tensorflow.keras.preprocessing.text import Tokenizer
def background_keyword(df): # input : spark dataframe을 pandas로 변환한 후, dataframe column
    background = df.values.tolist()
    token = Tokenizer()         # 토큰화 함수 지정
    token.fit_on_texts(background)    # 토큰화 함수에 문장 적용
    sorted_dic4 = sorted(token.word_counts.items(), key=lambda x: x[1], reverse=True)
    word_counts_1 = {}
    for i in sorted_dic4:
        word_counts_1.update({i[0]:i[1]})
    return word_counts_1 # dictionary 임
def back_keyword(pos,neg): # pos : event-related keyword, neg : event-unrelated keyword
    stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
    stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')
    word1 = stage_1.transform(pos) #나는 first_train 부터 함!
    word1 = stage_2.transform(word1)
    word2 = stage_1.transform(neg) #나는 first_train 부터 함!
    word2 = stage_2.transform(word2)
    from pyspark.sql.functions import col, concat_ws
    df1 = word1.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word1 = df1.select("filtered_words").toPandas()
    df2 = word2.withColumn("filtered_words",
       concat_ws(",",col("filtered_words")))
    word2 = df2.select("filtered_words").toPandas()
    word_counts_1 = background_keyword(word1["filtered_words"])
    word_counts_2 = background_keyword(word2["filtered_words"])
    return word_counts_1, word_counts_2




# Author : Minseon Kim (2021)
class BroadcastWrapper(object):
    def __init__(self, data, token_list):
        self.broadcast_var = sc.broadcast(data)
#         self.last_updated_time = datetime.now()
        self.token_list = token_list
    
#     def is_should_be_updated(self, data):
#         cur_time = datetime.now()
#         diff_sec = (cur_time - self.last_updated_time).total_seconds()
#         return self.broadcast_var is None or diff_sec> 1
    
    def update_and_get_data(self, spark):
        a = self.broadcast_var.value
        self.broadcast_var.unpersist()
        for i in self.token_list:
            for j in i:
                if j not in a.keys():
                    a[j] = 1
                else:
                    a[j] += 1
        new_data = a
        self.broadcast_var = spark.broadcast(new_data)
#         self.last_updated_time = datetime.now()
#         return len(self.token_list)
        return self.broadcast_var

# 바꾼 버전
from pyspark.ml import Transformer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame
from pyspark.sql import Row
from pyspark.sql import types   

def series_to_list(x):
    seri = x.values.tolist()
    for i in range(len(seri)):
        seri[i] = seri[i].tolist()
    return seri
class TextToSequence(Transformer):
    w1 = dict()
    w2 = dict()
    pos_t = Tokenizer(lower=False)
    neg_t = Tokenizer(lower=False)
    pos_vocab = []
    neg_vocab = []
    
    def __init__(self, w1: Dict[str, int], w2:Dict[str, int]):
        super(TextToSequence, self).__init__()
        self.w1 = w1
        self.w2 = w2
        
    
    def _transform(self, df: DataFrame):
        result_pdf = df.select("filtered_words").toPandas()
        pos_df = result_pdf[:500000] # 24939 99756
        neg_df = result_pdf[500000:]
        print(len(result_pdf))
        
        broadcast_wrapper1 = BroadcastWrapper(self.w1, pos_df["filtered_words"])
        ww1 = broadcast_wrapper1.update_and_get_data(sc).value
        
        broadcast_wrapper2 = BroadcastWrapper(self.w2, neg_df["filtered_words"])
        ww2 = broadcast_wrapper2.update_and_get_data(sc).value
        
        www1 = {k: v for k, v in sorted(ww1.items(), key=lambda item: item[1], reverse=True)}
        www2 = {k: v for k, v in sorted(ww2.items(), key=lambda item: item[1], reverse=True)}
        print(www1['allow'])
        print(www2['like'])
        aa = [w for i, w in enumerate(www1) if i < 5000]
        bb = [w for i, w in enumerate(www2) if i < 5000]
        
        self.pos_t.fit_on_texts(aa)
        self.neg_t.fit_on_texts(bb)
#         encoded_docs_pos = self.pos_t.texts_to_sequences(df.select("filtered_words1").toPandas())
#         encoded_docs_neg = self.neg_t.texts_to_sequences(df.select("filtered_words1").toPandas())
        result_list = series_to_list(result_pdf["filtered_words"])
        encoded_docs_pos = self.pos_t.texts_to_sequences(result_list) # 상위 event keyword 5k가 쓰인 tokenizer
        encoded_docs_neg = self.neg_t.texts_to_sequences(result_list)
        
        X_p = pad_sequences(encoded_docs_pos, maxlen=100, padding='post').tolist()
        X_n = pad_sequences(encoded_docs_neg, maxlen=100, padding='post').tolist()
        
        text_array = [str(row['text']) for row in df.select('text').collect()]
        label_array = [int(row['label']) for row in df.select('label').collect()]

        
        zip_array = list(zip(text_array, label_array, X_p, X_n))
        rdd = sc.parallelize(zip_array, numSlices=306)
        fdf = rdd.toDF(['text','label','feature1','feature2'])
        

        print(type(fdf))

        return fdf, www1, aa
    
    
word_counts_1, word_counts_2 = back_keyword(t05, t9)
word_counts_1 = sc.broadcast(word_counts_1)
word_counts_2 = sc.broadcast(word_counts_2)

stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')

r1 = stage_1.transform(fifth) #나는 first_train 부터 함!
r2 = stage_2.transform(r1)


# USE THE TRANSFORMER WITHOUT PIPELINE
text_sequence = TextToSequence(w1 = word_counts_1.value, w2 = word_counts_2.value)
df_example, www2, bb = text_sequence.transform(r2)
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df_with_vectors = df_example.select(
    df_example["label"], 
    list_to_vector_udf(df_example["feature1"]).alias("feature1"), 
    list_to_vector_udf(df_example["feature2"]).alias("feature2")
)
label_str_index = StringIndexer(inputCol='label', outputCol='label_index') # transformer의 마지막 단계!!
label_df2 = label_str_index.fit(df_with_vectors).transform(df_with_vectors)


1000000
314
81700
<class 'pyspark.sql.dataframe.DataFrame'>


In [24]:
trainqq, testqq = label_df.randomSplit(weights=[0.8,0.2]) # first train

In [25]:
trainqq1, testqq1 = label_df2.randomSplit(weights=[0.8,0.2]) # fifth window (no finetuning)

In [27]:
trainqq.toPandas().to_csv("/root/spark/bigcomp_model/typoon/first_train.csv",index=False)

/root/spark/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [28]:
testqq.toPandas().to_csv("/root/spark/bigcomp_model/typoon/first_test.csv",index=False)

In [29]:
trainqq1.toPandas().to_csv("/root/spark/bigcomp_model/typoon/fifth_tr2.csv",index=False)

In [30]:
testqq1.toPandas().to_csv("/root/spark/bigcomp_model/typoon/fifth_ts2.csv",index=False)

# 데이터프레임

In [18]:
trainqq= pd.read_csv("/root/spark/bigcomp_model/typoon/first_train.csv", sep=',',  lineterminator='\n')
trainqq = spark.createDataFrame(trainqq[['label','feature1','feature2','label_index']])
trainqq.show(5)
testqq= pd.read_csv("/root/spark/bigcomp_model/typoon/first_test.csv", sep=',',  lineterminator='\n')
testqq = spark.createDataFrame(testqq[['label','feature1','feature2','label_index']])
testqq.show(5)
trainqq1 = pd.read_csv("/root/spark/bigcomp_model/typoon/fifth_tr2.csv", sep=',',  lineterminator='\n')
trainqq1 = spark.createDataFrame(trainqq1[['label','feature1','feature2','label_index']])
testqq1 = pd.read_csv("/root/spark/bigcomp_model/typoon/fifth_ts2.csv", sep=',',  lineterminator='\n')
testqq1 = spark.createDataFrame(testqq1[['label','feature1','feature2','label_index']])

/root/spark/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  An error occurred while calling z:org.apache.spark.sql.api.python.PythonSQLUtils.readArrowStreamFromFile.
: java.lang.IllegalArgumentException
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:334)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.readMessage(MessageSerializer.java:543)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.readNextBatch(ArrowConverters.scala:243)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.<init>(ArrowConverters.scala:229)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$.getBatchesFromStream(ArrowConverters.scala:228)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun$readArrowStreamFromFile$2.apply(ArrowConverters.scala:216)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anonfun

+-----+--------------------+--------------------+-----------+
|label|            feature1|            feature2|label_index|
+-----+--------------------+--------------------+-----------+
|    1|[1.0,8.0,71.0,1.0...|[288.0,552.0,3346...|        1.0|
|    1|[1.0,8.0,129.0,10...|[288.0,692.0,122....|        1.0|
|    1|[1.0,42.0,742.0,1...|[3779.0,3148.0,1....|        1.0|
|    1|[1.0,44.0,12.0,87...|[5.0,4857.0,89.0,...|        1.0|
|    1|[1.0,44.0,12.0,87...|[5.0,4857.0,89.0,...|        1.0|
+-----+--------------------+--------------------+-----------+
only showing top 5 rows

+-----+--------------------+--------------------+-----------+
|label|            feature1|            feature2|label_index|
+-----+--------------------+--------------------+-----------+
|    1|[1.0,8.0,66.0,69....|[288.0,138.0,64.0...|        1.0|
|    1|[1.0,8.0,1902.0,1...|[288.0,2569.0,555...|        1.0|
|    1|[1.0,44.0,574.0,1...|[544.0,4993.0,796...|        1.0|
|    1|[1.0,44.0,1534.0,...|[138.0,64.0,311.0

In [19]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
import pyspark.sql.types as T

parse = udf(lambda l: Vectors.dense(l), VectorUDT())

import pyspark.sql.functions as F
import pyspark.sql.types as T
import json

def parse_embedding_from_string(x):
    res = json.loads(x)
    return res

retrieve_embedding = F.udf(parse_embedding_from_string, T.ArrayType(T.LongType()))

trainqq = trainqq.select(
    trainqq["label_index"], 
    retrieve_embedding(trainqq["feature1"]).alias("feature1"),
    retrieve_embedding(trainqq["feature2"]).alias("feature2"),
    trainqq["label"]
)
trainqq = trainqq.select(
    trainqq["label_index"], 
    parse(trainqq["feature1"]).alias("feature1"),
    parse(trainqq["feature2"]).alias("feature2"),
    trainqq["label"]
)



trainqq1 = trainqq1.select(
    trainqq1["label_index"], 
    retrieve_embedding(trainqq1["feature1"]).alias("feature1"),
    retrieve_embedding(trainqq1["feature2"]).alias("feature2"),
    trainqq1["label"]
)
trainqq1 = trainqq1.select(
    trainqq1["label_index"], 
    parse(trainqq1["feature1"]).alias("feature1"),
    parse(trainqq1["feature2"]).alias("feature2"),
    trainqq1["label"]
)


In [20]:
testqq = testqq.select(
    testqq["label_index"], 
    retrieve_embedding(testqq["feature1"]).alias("feature1"),
    retrieve_embedding(testqq["feature2"]).alias("feature2"),
    testqq["label"]
)
testqq = testqq.select(
    testqq["label_index"], 
    parse(testqq["feature1"]).alias("feature1"),
    parse(testqq["feature2"]).alias("feature2"),
    testqq["label"]
)
testqq1 = testqq1.select(
    testqq1["label_index"], 
    retrieve_embedding(testqq1["feature1"]).alias("feature1"),
    retrieve_embedding(testqq1["feature2"]).alias("feature2"),
    testqq1["label"]
)
testqq1 = testqq1.select(
    testqq1["label_index"], 
    parse(testqq1["feature1"]).alias("feature1"),
    parse(testqq1["feature2"]).alias("feature2"),
    testqq1["label"]
)

# Classifier

In [21]:
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, concatenate
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers import Conv1D, Flatten, GlobalMaxPooling1D, Dropout, MaxPooling1D, Activation, Bidirectional, Conv2D, GlobalMaxPooling2D
nb_classes=2
vocab_sizes = 5001
neg_vocab_sizes = 5001
embedding_vector_length = 100
hidden_dims = 250
max_length = 100
dropout_ratio = 0.5
from tensorflow.keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_sizes, embedding_vector_length, input_length=max_length))

#어휘 크기(최대 정수 인덱스+1, 고밀도 임베딩의 치수, 입력 시퀀스의 길이

# 모델은 크기의 정수 행렬 (배치,
# input_length) 및 입력에서 가장 큰 정수 (즉, 단어 인덱스)
#은 999 (어휘 크기)보다 크지 않아야합니다.
# 이제 model.output_shape는 (None, 10, 64)이고, 여기서`None`은 배치입니다.
# dimension.
model.add(Dropout(dropout_ratio))
model.add(Conv1D(256, 5, activation='relu'))
#model.add(Bidirectional(LSTM(128)))
model.add(GlobalMaxPooling1D())
model.add(Dense(250, activation='relu')) # 한 층 더 쌓을까..? relu 함수는 정류된 함수로 은닉층에서 많이 사용된다. -를 차단함!
# 은닉층으로 많이 사용되는 이유는 음수는 0으로 반환하므로 특정 양수값으로 수렴하지 않음. 그래서 기울기 소실이 발생하지 않음!
# 시그모이드 함수는 기울기 소실이 발생한다는 단점이 존재. 그래서 relu를 은닉층으로 쓰는 거임!
# 또한, 학습 속도가 매우 빠르다.
model.add(Dropout(dropout_ratio))
model.add(Dense(nb_classes)) # stringIndexer 클래스 수만큼 쌓는 이유가 뭘까,,,
model.add(Activation('sigmoid'))
#model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
"""#model.add(Dropout(dropout_ratio))
model.add(Conv1D(256, 3,  padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
#model.add(Dense(128, activation='relu')) #
model.add(Dropout(dropout_ratio))
model.add(Activation('relu'))
model.add(Dense(nb_classes)) # stringIndexer 클래스 수만큼 쌓는 이유가 뭘까,,,
model.add(Activation('sigmoid'))
#model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])"""

"""model.add(Bidirectional(LSTM(128)))
# model.add(GlobalMaxPooling1D())
model.add(Activation('relu')) # 한 층 더 쌓을까..? relu 함수는 정류된 함수로 은닉층에서 많이 사용된다. -를 차단함!
# 은닉층으로 많이 사용되는 이유는 음수는 0으로 반환하므로 특정 양수값으로 수렴하지 않음. 그래서 기울기 소실이 발생하지 않음!
# 시그모이드 함수는 기울기 소실이 발생한다는 단점이 존재. 그래서 relu를 은닉층으로 쓰는 거임!
# 또한, 학습 속도가 매우 빠르다."""
"""model.add(Dense(nb_classes)) # stringIndexer 클래스 수만큼 쌓는 이유가 뭘까,,,
model.add(Activation('sigmoid'))
# model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])"""

# #체크포인트 생성?!
# # checkpoint_filepath = '/root/spark/baseline/'
# fname = "checkpoint-{epoch:02d}-{val_loss:.2f}"
# checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min",
#     save_best_only=True, verbose=1)
# callbacks = [checkpoint]

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          500100    
_________________________________________________________________
dropout (Dropout)            (None, 100, 100)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 96, 256)           128256    
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 250)               64250     
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 5

In [36]:
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, concatenate
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers import Conv1D, Flatten, GlobalMaxPooling1D, Dropout, MaxPooling1D, Activation, Bidirectional, Conv2D, GlobalMaxPooling2D
nb_classes=2
vocab_sizes = 5001
neg_vocab_sizes = 5001
embedding_vector_length = 100
hidden_dims = 250
max_length = 100
dropout_ratio = 0.5
from tensorflow.keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_sizes, embedding_vector_length, input_length=max_length))

#어휘 크기(최대 정수 인덱스+1, 고밀도 임베딩의 치수, 입력 시퀀스의 길이

# 모델은 크기의 정수 행렬 (배치,
# input_length) 및 입력에서 가장 큰 정수 (즉, 단어 인덱스)
#은 999 (어휘 크기)보다 크지 않아야합니다.
# 이제 model.output_shape는 (None, 10, 64)이고, 여기서`None`은 배치입니다.
# dimension.
"""model.add(Dropout(dropout_ratio))
model.add(Conv1D(256, 5, activation='relu'))
#model.add(Bidirectional(LSTM(128)))
model.add(GlobalMaxPooling1D())
model.add(Dense(250, activation='relu')) # 한 층 더 쌓을까..? relu 함수는 정류된 함수로 은닉층에서 많이 사용된다. -를 차단함!
# 은닉층으로 많이 사용되는 이유는 음수는 0으로 반환하므로 특정 양수값으로 수렴하지 않음. 그래서 기울기 소실이 발생하지 않음!
# 시그모이드 함수는 기울기 소실이 발생한다는 단점이 존재. 그래서 relu를 은닉층으로 쓰는 거임!
# 또한, 학습 속도가 매우 빠르다.
model.add(Dropout(dropout_ratio))
model.add(Dense(nb_classes)) # stringIndexer 클래스 수만큼 쌓는 이유가 뭘까,,,
model.add(Activation('sigmoid'))
#model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])"""
"""#model.add(Dropout(dropout_ratio))
model.add(Conv1D(256, 3,  padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
#model.add(Dense(128, activation='relu')) #
model.add(Dropout(dropout_ratio))
model.add(Activation('relu'))
model.add(Dense(nb_classes)) # stringIndexer 클래스 수만큼 쌓는 이유가 뭘까,,,
model.add(Activation('sigmoid'))
#model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])"""

model.add(Bidirectional(LSTM(128)))
# model.add(GlobalMaxPooling1D())
model.add(Activation('relu')) # 한 층 더 쌓을까..? relu 함수는 정류된 함수로 은닉층에서 많이 사용된다. -를 차단함!
# 은닉층으로 많이 사용되는 이유는 음수는 0으로 반환하므로 특정 양수값으로 수렴하지 않음. 그래서 기울기 소실이 발생하지 않음!
# 시그모이드 함수는 기울기 소실이 발생한다는 단점이 존재. 그래서 relu를 은닉층으로 쓰는 거임!
# 또한, 학습 속도가 매우 빠르다.
model.add(Dense(nb_classes)) # stringIndexer 클래스 수만큼 쌓는 이유가 뭘까,,,
model.add(Activation('sigmoid'))
# model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# #체크포인트 생성?!
# # checkpoint_filepath = '/root/spark/baseline/'
# fname = "checkpoint-{epoch:02d}-{val_loss:.2f}"
# checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min",
#     save_best_only=True, verbose=1)
# callbacks = [checkpoint]

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          500100    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               234496    
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
_________________________________________________________________
activation_3 (Activation)    (None, 2)                 0         
Total params: 735,110
Trainable params: 735,110
Non-trainable params: 0
_________________________________________________________________
None


# fine tuning

In [20]:
from tensorflow.keras.models import load_model
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, concatenate
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers import Conv1D, Flatten, GlobalMaxPooling1D, Dropout, MaxPooling1D, Activation, Bidirectional, Conv2D, GlobalMaxPooling2D
model1 = load_model('./bigcomp_model/model/typretrained_cnn.h5')
print(model1.summary())
model1.layers[0].trainable = False
model1.layers[1].trainable = False
model1.layers[2].trainable = False
model1.layers[3].trainable = False
model1.layers[4].trainable = False
model1.layers[5].trainable = False
trans_model = Sequential(model1.layers[:6])
trans_model.add(Dense(2))
trans_model.add(Activation('sigmoid'))
trans_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(trans_model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          500100    
_________________________________________________________________
dropout (Dropout)            (None, 100, 100)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 98, 256)           77056     
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [21]:
from tensorflow.keras.models import load_model
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, concatenate
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers import Conv1D, Flatten, GlobalMaxPooling1D, Dropout, MaxPooling1D, Activation, Bidirectional, Conv2D, GlobalMaxPooling2D
model1 = load_model('./bigcomp_model/model/typretrained_rnn.h5')
print(model1.summary())
model1.layers[0].trainable = False
model1.layers[1].trainable = False
trans_model = Sequential(model1.layers[:2])
trans_model.add(Dense(2))
trans_model.add(Activation('sigmoid'))
trans_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(trans_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          500100    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                4256      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 504,389
Trainable params: 504,389
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          500100    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                4256      
_______________________

In [68]:
model1.layers

In [22]:
from keras import optimizers, regularizers
from tensorflow.keras import optimizers

optimizer_conf = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)

In [23]:
from tensorflow.keras.models import model_from_json
from elephas import spark_model, ml_model
from elephas.ml_model import ElephasEstimator

estimator = ElephasEstimator()
estimator.setFeaturesCol("feature1")
estimator.setLabelCol("label_index")
#estimator.set_keras_model_config(trans_model.to_json())
estimator.set_keras_model_config(model.to_json())
estimator.set_categorical_labels(True) # dense 1이면 False로 설정해야함
estimator.set_nb_classes(2)
estimator.set_num_workers(3)
estimator.set_epochs(10)
estimator.set_batch_size(128)
estimator.set_verbosity(1)
estimator.set_validation_split(0.10)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("asynchronous")
estimator.set_loss("binary_crossentropy")
estimator.set_metrics(['acc'])

ElephasEstimator_9759056e21ee

In [24]:
dl_pipeline = Pipeline(stages=[estimator])

In [24]:
trainqq.show(10)

+-----------+--------------------+--------------------+-----+
|label_index|            feature1|            feature2|label|
+-----------+--------------------+--------------------+-----+
|        1.0|[1.0,8.0,71.0,1.0...|[288.0,552.0,3346...|    1|
|        1.0|[1.0,8.0,129.0,10...|[288.0,692.0,122....|    1|
|        1.0|[1.0,42.0,742.0,1...|[3779.0,3148.0,1....|    1|
|        1.0|[1.0,44.0,12.0,87...|[5.0,4857.0,89.0,...|    1|
|        1.0|[1.0,44.0,12.0,87...|[5.0,4857.0,89.0,...|    1|
|        1.0|[1.0,44.0,31.0,20...|[16.0,81.0,755.0,...|    1|
|        1.0|[1.0,44.0,66.0,69...|[138.0,64.0,3075....|    1|
|        1.0|[1.0,44.0,93.0,15...|[108.0,1672.0,311...|    1|
|        1.0|[1.0,44.0,399.0,7...|[2641.0,79.0,4345...|    1|
|        1.0|[1.0,44.0,570.0,1...|[710.0,113.0,0.0,...|    1|
+-----------+--------------------+--------------------+-----+
only showing top 10 rows



In [25]:
import time
t1 = time.time()
my_dl = dl_pipeline.fit(trainqq) # cnn first train (not finetuning)
t3 = time.time() - t1
print(t3)
aaaaaa = "완료!!"

>>> Fit model
 * Serving Flask app 'elephas.parameter.server' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://172.28.0.1:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


172.28.0.1 - - [12/Oct/2022 18:58:29] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 18:58:29] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 18:58:29] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 18:58:50] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 18:58:50] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 18:58:50] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 18:58:50] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 18:58:51] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 18:58:51] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 18:59:11] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 18:59:11] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 18:59:11] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 18:59:11] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 18:59:12] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2

>>> Async training complete.
255.6982741355896


In [26]:
cs_model = my_dl.stages[0]
print(cs_model)

cs_model.save("/root/spark/bigcomp_model/model/typoon/cnn/first2")

ElephasTransformer_557762362fc8


In [27]:
import time
t1 = time.time()
my_dl1 = dl_pipeline.fit(trainqq1) # cnn 5th not finetuning
t3 = time.time() - t1
print(t3)
aaaaaa = "완료!!"

>>> Fit model
 * Serving Flask app 'elephas.parameter.server' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://172.28.0.1:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


172.28.0.1 - - [12/Oct/2022 19:06:11] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 19:07:03] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 19:07:06] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 19:10:13] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 19:10:13] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 19:10:41] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 19:10:41] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 19:10:41] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 19:10:41] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 19:13:45] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 19:13:45] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 19:14:13] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 19:14:13] "GET /parameters HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2022 19:14:14] "POST /update HTTP/1.1" 200 -
172.28.0.1 - - [12/Oct/2

>>> Async training complete.
2416.637028694153


In [28]:
cs_model = my_dl1.stages[0]
print(cs_model)

cs_model.save("/root/spark/bigcomp_model/model/typoon/cnn/fine5")

ElephasTransformer_6abbb57d10d0


In [29]:


from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics

#pred_test = my_dl.transform(label_dft)
pred_test = my_dl.transform(testqq)
from pyspark.sql.functions import udf
list_to_double = udf(lambda l: l[0], DoubleType())
pred_test = pred_test.select(
    pred_test["label_index"], 
    list_to_double(pred_test["prediction"]).alias("prediction2")
)
pred_test.printSchema()

from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l:1.0 if l<0.5 else 0.0 , DoubleType())
pred_test = pred_test.select(
    pred_test["label_index"],
    pred_test["prediction2"],
    list_to_vector_udf(pred_test["prediction2"]).alias("rawPrediction")
)
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
dfdf1 = pred_test.toPandas() # fine-tuned RNN 2nd window


root
 |-- label_index: double (nullable = true)
 |-- prediction2: double (nullable = true)



In [41]:
import time
t1 = time.time()
my_dl = dl_pipeline.fit(trainqq1)
t3 = time.time() - t1
print(t3)
aaaaaa = "완료!!"

>>> Fit model
>>> Initialize workers
>>> Distribute load
 * Serving Flask app 'elephas.parameter.server' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://172.28.0.1:4000/ (Press CTRL+C to quit)
INFO:werkzeug:172.28.0.2 - - [05/Oct/2022 08:17:59] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [05/Oct/2022 08:18:02] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.3 - - [05/Oct/2022 08:18:03] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.2 - - [05/Oct/2022 08:19:03] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [05/Oct/2022 08:19:12] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.3 - - [05/Oct/2022 08:19:24] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [05/Oct/2022 08:19:46] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [05/Oct/2022 08:36:49] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [05/Oct/2022 08:36:50] "GET /parameters HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [05/Oct/2022 08:40:21] "POST /update HTTP/1.1" 200 -
INFO:werkzeug:172.28.0.1 - - [05/Oct/2022 08:40:21] "GET /parameters HTTP/1.1" 20

>>> Async training complete.


INFO:werkzeug:172.28.0.1 - - [05/Oct/2022 11:47:46] "GET /parameters HTTP/1.1" 200 -


14590.308893918991


In [42]:
cs_model = my_dl.stages[0]
print(cs_model)

cs_model.save("/root/spark/bigcomp_model/model/typoon/lstm/fifth")

ElephasTransformer_c1d598385fd1


In [43]:
pred_test = my_dl.transform(testqq1)

In [27]:
pred_test.show(5)

+-----------+--------------------+--------------------+-----+----------+
|label_index|            feature1|            feature2|label|prediction|
+-----------+--------------------+--------------------+-----+----------+
|        1.0|[1.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|    1|[0.0, 1.0]|
|        1.0|[1.0,0.0,0.0,0.0,...|[4001.0,4586.0,0....|    1|[0.0, 1.0]|
|        1.0|[1.0,10.0,1.0,177...|[333.0,141.0,65.0...|    1|[0.0, 1.0]|
|        1.0|[1.0,10.0,160.0,1...|[333.0,141.0,65.0...|    1|[0.0, 1.0]|
|        1.0|[1.0,10.0,1920.0,...|[333.0,2129.0,508...|    1|[0.0, 1.0]|
+-----------+--------------------+--------------------+-----+----------+
only showing top 5 rows



In [34]:
print("타이푼 finetuning second : ",t3)

타이푼 finetuning second :  2257.6532146930695


In [55]:
print("타이푼 third : ",t3)

타이푼 third :  7631.656853199005


In [66]:
print("타이푼 fourth : ",t3)

타이푼 fourth :  10176.846570968628


In [ ]:
trainqq.toPandas().to_csv("/root/spark/bigcomp_model/typoon/third_tr.csv",index=False)
testqq.toPandas().to_csv("/root/spark/bigcomp_model/typoon/third_ts.csv",index=False)

In [44]:


from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics

#pred_test = my_dl.transform(label_dft)
pred_test = my_dl.transform(testqq1)
from pyspark.sql.functions import udf
list_to_double = udf(lambda l: l[0], DoubleType())
pred_test = pred_test.select(
    pred_test["label_index"], 
    list_to_double(pred_test["prediction"]).alias("prediction2")
)
pred_test.printSchema()

from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l:1.0 if l<0.5 else 0.0 , DoubleType())
pred_test = pred_test.select(
    pred_test["label_index"],
    pred_test["prediction2"],
    list_to_vector_udf(pred_test["prediction2"]).alias("rawPrediction")
)


root
 |-- label_index: double (nullable = false)
 |-- prediction2: double (nullable = true)



In [ ]:
pred_test.show(50)